# Introduction

This notebook is to study the DENTIST algorithm based on the original C++ [(github repo)](https://github.com/Yves-CHEN/DENTIST) and the version we converted into R [slalom.R](https://github.com/cumc/pecotmr/blob/main/R/slalom.R).

# DENTIST in C++

## Installation

Please follow the instructions from [Gao's fork](https://github.com/gaow/DENTIST/tree/master). We failed to compile the original repo.

The compiled executable file will be stored in `./builts` folder.

## Run DENTIST on test data

```shell
/home/rd2972/software/DENTIST/builts/DENTIST.tmp2 \
    --bfile  /home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_input_MWE \
    --gwas-summary /home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/Sumstat_MAF_filtered.txt \
    --out /home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/out
```

- `Input data`
    - The bfiles contain 4123 variants of 1153 individuals.
    - The summary statistics is for the same 4123 variants.
- `Output results`
    - Output from the original `DENTIST`
        - `/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/out.DENTIST.full.txt` (4123 variants)
        - `/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/out.DENTIST.ignored.txt` (empty)
        - `/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/out.DENTIST.short.txt` (63 variants)
    - Output of `LD_it`:
        - We edit the C++ code so every time it runs `oneIteration` function, it will append `LD_it` to `/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/LD_it_output.txt`
        - Then we separate the output file into seperate `LD_it`, stored at `/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/LD_separate/`
        - **WEIRD WEIRD WEIRD**: for 8 iterations there are 32 `LD_it`s. (why not 16??)

# Run dentist in R

In [1]:
rm(list=ls())
library(readr)
library(dplyr, warn.conflicts = FALSE)
library(data.table)
library(RcppArmadillo)
library(Rcpp)
source("/home/rd2972/software/pecotmr/R/run_dentist.R")
source("/home/rd2972/software/pecotmr/R/RcppExports.R")
sourceCpp("/home/rd2972/software/pecotmr/src/dentist.cpp")
sourceCpp("/home/rd2972/software/pecotmr/src/RcppExports.cpp")


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


Registered S3 methods overwritten by 'RcppGSL':
  method               from         
  predict.fastLm       RcppArmadillo
  print.fastLm         RcppArmadillo
  summary.fastLm       RcppArmadillo
  print.summary.fastLm RcppArmadillo

Warning message in sourceCpp("/home/rd2972/software/pecotmr/src/RcppExports.cpp"):
“No Rcpp::export attributes or RCPP_MODULE declarations found in source”


In [2]:
snp_data = read_delim("/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/Sumstat_MAF_filtered.txt", show_col_types = FALSE) %>% mutate(z = b / se)
LD_data = fread("/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/LD_MWE.tsv")[,-1]
LD_data = as.matrix(LD_data)

Warning message in fread("/home/hs3393/RSS_QC/pecotmr/data/RSS_QC_MWE/LD_MWE.tsv"):
“Detected 4123 column names but the data has 4124 columns (i.e. invalid file). Added 1 extra default column name for the first column which is guessed to be row names or an index. Use setnames() afterwards if this guess is not correct, or fix the file write command that created the file to create a valid file.”


In [3]:
head(snp_data,3)
dim(snp_data)

SNP,A1,A2,freq,b,se,p,N,z
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1:206011748_A_G,G,A,0.1865,-0.0169,0.0115,0.07083977,1153,-1.469565
chr1:206012565_A_G,G,A,0.1908,-0.0189,0.0114,0.04866936,1153,-1.657895
chr1:206012721_C_G,G,C,0.1930,-0.0185,0.0114,0.05231533,1153,-1.622807


[1] 4123    9

In [4]:
res1 = dentist(snp_data$z, LD_data, 
                 nSample = 1153, nIter = 8, 
                 pValueThreshold = 5.0369e-8, gPvalueThreshold = 0.05,
                 propSVD = 0.4, gcControl = FALSE)
# most argument values are matched with default values here: /home/rd2972/software/DENTIST/options.h
# 1153 comes from ` /home/hs3393//RSS_QC/pecotmr/data/RSS_QC_MWE/PLINK_input_MWE.fam`
# the output are all stored at `/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/`

In [5]:
head(res1,3)

,imputed_z,rsq,corrected_z,iter_to_correct,is_problematic,original_z
,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
1,-0.3698164,0.7568408,-2.230222,8,0,-1.469565
2,-0.2682808,0.7585386,-2.827941,8,0,-1.657895
3,-0.2928419,0.7613557,-2.722480,8,0,-1.622807


In [6]:
saveRDS(res1, file = "/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/result.rds")

Note: There are two `LD_it`s for each iteration, and the index of iteration starts from 0. For example, `DENTIST_Rcpp_output_42.txt` stores the `LD_it` from the second time it runs `oneIteration` function in the No.5 iteration.

# Compare results

In [8]:
# load the two LD_its from the first ieration
Rcpp_res_iter_0_1 <- fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/4213_variants/DENTIST_Rcpp_output_01.txt")
Rcpp_res_iter_0_2 <- fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_Rcpp/4213_variants/DENTIST_Rcpp_output_02.txt")

In [9]:
# load the first two LD_its from 
dentist_res_1 <-fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/LD_separate/LD_it_output_1.txt")
dentist_res_2 <-fread("/home/rd2972/private_data/20240300_rss_qc_imputation/DENTIST/per_iteration/dentist_original/4213_variants/LD_separate/LD_it_output_2.txt")

In [12]:
print("==============From Rcpp version=====================")
dim(Rcpp_res_iter_0_1)
Rcpp_res_iter_0_1[1:10,1:10]
dim(Rcpp_res_iter_0_2)
Rcpp_res_iter_0_2[1:10,1:10]
print("==============From DENTIST C++ version=====================")
dim(dentist_res_1)
dentist_res_1[1:10,1:10]
dim(dentist_res_2)
dentist_res_2[1:10,1:10]

[1] "==============From Rcpp version====================="


[1] 2062 2061

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.98501,0.99703,-0.165970,0.98062,0.95174,0.305990,0.196350,0.24287000,1.9603e-01,0.200050
0.97722,0.98368,-0.165530,0.99097,0.96237,0.309330,0.182120,0.24568000,1.9891e-01,0.184900
0.94805,0.94244,-0.078334,0.96493,0.99127,0.299980,0.174640,0.23784000,1.9103e-01,0.177670
0.41409,0.40288,-0.056151,0.41477,0.39906,-0.034997,-0.049078,-0.02528900,-3.2538e-02,-0.046423
0.58251,0.57434,-0.182270,0.58924,0.60417,0.173310,0.068187,0.16826000,8.4623e-02,0.077967
0.28936,0.28088,-0.048627,0.29017,0.27282,0.900440,0.708410,-0.03901300,-7.5352e-05,0.724340
0.19780,0.21003,-0.066790,0.19857,0.18544,0.633320,0.938430,-0.00181520,1.8330e-02,0.987060
0.18323,0.21323,-0.064954,0.18401,0.17146,0.590860,0.938180,-0.00176530,1.7826e-02,0.986700
0.18698,0.21642,-0.066252,0.18776,0.17500,0.607260,0.950820,-0.00257810,1.6673e-02,1.000000


[1] 2053 2061

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.98501,0.99703,-0.165970,0.98062,0.95174,0.305990,0.196350,0.24287000,1.9603e-01,0.200050
0.97722,0.98368,-0.165530,0.99097,0.96237,0.309330,0.182120,0.24568000,1.9891e-01,0.184900
0.94805,0.94244,-0.078334,0.96493,0.99127,0.299980,0.174640,0.23784000,1.9103e-01,0.177670
0.41409,0.40288,-0.056151,0.41477,0.39906,-0.034997,-0.049078,-0.02528900,-3.2538e-02,-0.046423
0.58251,0.57434,-0.182270,0.58924,0.60417,0.173310,0.068187,0.16826000,8.4623e-02,0.077967
0.28936,0.28088,-0.048627,0.29017,0.27282,0.900440,0.708410,-0.03901300,-7.5352e-05,0.724340
0.19780,0.21003,-0.066790,0.19857,0.18544,0.633320,0.938430,-0.00181520,1.8330e-02,0.987060
0.18323,0.21323,-0.064954,0.18401,0.17146,0.590860,0.938180,-0.00176530,1.7826e-02,0.986700
0.18698,0.21642,-0.066252,0.18776,0.17500,0.607260,0.950820,-0.00257810,1.6673e-02,1.000000


[1] "==============From DENTIST C++ version====================="


[1] 828 827

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.948033,0.948211,0.955779,-0.0785996,0.964921,0.991271,0.4037350,0.1881340,0.1776080,0.237791000
0.312609,0.305939,0.309283,-0.0338847,0.313154,0.296342,-0.0350442,0.6333060,0.6072500,-0.035160200
0.582866,0.575675,0.585092,-0.1821460,0.589592,0.604548,0.2425070,0.0866302,0.0780352,0.168334000
0.289292,0.282638,0.286153,-0.0487172,0.290100,0.272754,-0.0425133,0.7553270,0.7243300,-0.039047800
0.183171,0.196556,0.181131,-0.0650268,0.183950,0.171392,-0.0457802,0.9725030,0.9867000,-0.001791760
0.166918,0.196288,0.182052,-0.0713314,0.185002,0.171811,-0.0491233,0.9384300,0.9508180,-0.005674030
0.192370,0.195959,0.198841,0.0278439,0.201963,0.188044,-0.0325835,0.0183002,0.0166432,-0.033922000
0.203609,0.198676,0.201369,-0.0655563,0.204336,0.191198,-0.0463954,0.9855720,0.9456780,-0.000985741
0.183171,0.196556,0.181131,-0.0650268,0.183950,0.171392,-0.0457802,0.9725030,0.9867000,-0.001791760


[1] 827 824

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.9480330,0.3126090,0.5828660,0.2892920,0.18317100,0.16691800,0.1923700,0.203609000,0.18317100,0.16782300
0.9482110,0.3059390,0.5756750,0.2826380,0.19655600,0.19628800,0.1959590,0.198676000,0.19655600,0.19803300
0.9557790,0.3092830,0.5850920,0.2861530,0.18113100,0.18205200,0.1988410,0.201369000,0.18113100,0.18335100
-0.0785996,-0.0338847,-0.1821460,-0.0487172,-0.06502680,-0.07133140,0.0278439,-0.065556300,-0.06502680,-0.06886560
0.9649210,0.3131540,0.5895920,0.2901000,0.18395000,0.18500200,0.2019630,0.204336000,0.18395000,0.18626000
0.9912710,0.2963420,0.6045480,0.2727540,0.17139200,0.17181100,0.1880440,0.191198000,0.17139200,0.17328000
0.4037350,-0.0350442,0.2425070,-0.0425133,-0.04578020,-0.04912330,-0.0325835,-0.046395400,-0.04578020,-0.04781030
0.1881340,0.6333060,0.0866302,0.7553270,0.97250300,0.93843000,0.0183002,0.985572000,0.97250300,0.93498400
0.1776080,0.6072500,0.0780352,0.7243300,0.98670000,0.95081800,0.0166432,0.945678000,0.98670000,0.94870200
